In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import cv2 as cv
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from glob import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
df = pd.read_csv('/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df['age'].fillna(int(df['age'].mean()),inplace=True)

In [ ]:
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

In [ ]:
base_skin_dir = ('/kaggle/input/skin-cancer-mnist-ham10000')
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_skin_dir, '*', '*.jpg'))}

In [ ]:
df['path'] = df['image_id'].map(imageid_path_dict.get)
df['cell_type'] = df['dx'].map(lesion_type_dict.get)

In [ ]:
df['cell_type_idx'] = pd.Categorical(df['cell_type']).codes
df.head()

In [ ]:
from keras.preprocessing.image import img_to_array, array_to_img
df['image'] = df['path'].map(lambda x : img_to_array(cv.resize(cv.imread(x),(224,224),
                                                               interpolation=cv.INTER_AREA)))

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize = (10, 5))
df['cell_type'].value_counts().plot(kind='bar', ax=ax1)

In [ ]:
df2 = df[["image", 'cell_type_idx']]

In [ ]:
X=df2['image']
Y=df2['cell_type_idx']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state = 0) 

In [ ]:
x_train = np.asarray(x_train.tolist())
x_test = np.asarray(x_test.tolist())

In [ ]:
y_train = to_categorical(y_train, num_classes = 7)
y_test = to_categorical(y_test, num_classes = 7)

In [ ]:
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size = 0.1, random_state = 2)

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dropout, Dense, BatchNormalization
from tensorflow.keras.models import Model

# Image size and number of classes
IMG_SIZE = (224, 224, 3)
NUM_CLASSES = 7

# Step 1: Define the ResNet50 base model
base_model = ResNet50(include_top=False, weights="/kaggle/input/resnet-weights-2/tensorflow2/default/1/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5", input_shape=IMG_SIZE)

# Step 2: Add custom layers
inputs = Input(shape=IMG_SIZE)
x = base_model(inputs, training=False)  # Freeze base ResNet50
x = GlobalAveragePooling2D()(x)         # Global pooling
x = BatchNormalization()(x)             # Batch normalization
x = Dropout(0.5)(x)                     # Dropout for regularization
x = Dense(512, activation='relu')(x)    # Fully connected layer
x = Dropout(0.5)(x)                     # Dropout again
outputs = Dense(NUM_CLASSES, activation='softmax')(x)  # Output layer

# Create the model
model = Model(inputs, outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


In [ ]:
from tensorflow.keras.optimizers import Adam

train_datagen = ImageDataGenerator(
    horizontal_flip=True,
    preprocessing_function=tf.keras.applications.mobilenet.preprocess_input
)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

x_train_gen = train_datagen.flow(x_train, y_train, batch_size=16)
val_data = train_datagen.flow(x_validate, y_validate, batch_size=16)

In [ ]:
history = model.fit(
    x_train_gen,
    epochs=25,
    validation_data=val_data,
    batch_size=32  # Batch size should match the batch size in the data generator
)

In [ ]:
model.save('/kaggle/working/resnetModelHAM1000M.h5')
print("hello world")